# **XGBoost**
For classification of Processed Energy Dataset.

In [ ]:
# Import the standard tools for pythonic data analysis. 
import csv
import math
import random
import numpy as np 
import pandas as pd 


## Initial import and basic understanding of data

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Import the final dataset as dataframe from csv
df = pd.read_csv('/content/gdrive/MyDrive/CS3244 Team 05/Dataset Analysis/fixed_data_train.csv',sep=',',low_memory=False) # Separate on semicolons

In [ ]:
df.head()

,Unnamed: 0,client_id,district,client_catg,region,region_group,creation_date_day,creation_date_month,creation_date_year,no_months_as_client,...,tally_check_false,sum_tally_value,min_tally_value,max_tally_value,mean_tally_value,counter_type,last_year,last_month,last_day,last_day_is_weekday
0,0,train_Client_0,60,11,101,2,31,12,1994,290.568595,...,0,0,0,0,0.0,1,2019,3,19,1
1,1,train_Client_1,69,11,107,2,29,5,2002,202.123247,...,0,0,0,0,0.0,1,2019,4,2,1
2,2,train_Client_10,62,11,301,3,13,3,1986,397.642662,...,0,0,0,0,0.0,1,2019,5,2,1
3,3,train_Client_100,69,11,105,2,7,11,1996,190.591183,...,0,0,0,0,0.0,1,2012,9,25,1
4,4,train_Client_1000,62,11,303,3,14,10,2014,56.083287,...,0,0,0,0,0.0,1,2019,6,17,1


In [ ]:
#df.dtypes # Checking for data types

In [ ]:
#for i in df.columns:
  #print(str(i)+":"+str(df[i].isnull().values.any()))

## Data Cleaning or Selection

In [ ]:
del df['Unnamed: 0'] # Delete Index of CSV file
del df['client_id'] # Delete client_id as it is a string and can't be used as a feature

In [ ]:
df.head()

,district,client_catg,region,region_group,creation_date_day,creation_date_month,creation_date_year,no_months_as_client,services_consumed,target,...,tally_check_false,sum_tally_value,min_tally_value,max_tally_value,mean_tally_value,counter_type,last_year,last_month,last_day,last_day_is_weekday
0,60,11,101,2,31,12,1994,290.568595,1,0,...,0,0,0,0,0.0,1,2019,3,19,1
1,69,11,107,2,29,5,2002,202.123247,1,0,...,0,0,0,0,0.0,1,2019,4,2,1
2,62,11,301,3,13,3,1986,397.642662,1,0,...,0,0,0,0,0.0,1,2019,5,2,1
3,69,11,105,2,7,11,1996,190.591183,1,0,...,0,0,0,0,0.0,1,2012,9,25,1
4,62,11,303,3,14,10,2014,56.083287,1,0,...,0,0,0,0,0.0,1,2019,6,17,1


## Separating Training and Testing Data

Partition the dataset such that both training and testing data have a good mixed of fraud and no fraud data by randomness.

In [ ]:
from sklearn.model_selection import train_test_split

# Partion the features from the target to predict
df_features = df[df.columns[df.columns != 'target']].copy() # get columns that are not 'target'; this our features
df_target = df['target'].copy() # get the column named 'target'; this is our label

# (random_state): we use a fixed random seed so we get the same results every time.
X_train, X_test, y_train, y_test = train_test_split(df_features, df_target, test_size=0.3, random_state=1) ## RANDOM STATE DETERMINED HERE

print ("Number of training instances: ", len(X_train), "\nNumber of test instances: ", len(X_test), "\nTotal number of instances: ",len(df_target))

Number of training instances:  94840 
Number of test instances:  40647 
Total number of instances:  135487


## Random Undersampling


In [ ]:
# # Training Data
# not_fraud = y_train[y_train == 0].index #Get index of rows that are not fraud
# fraud = y_train[y_train == 1].index # #Get index of rows that are fraud
# not_fraud = np.random.choice(not_fraud, len(fraud), replace=False) # Randomly pick not fraud rows without replacement until the len is the same as fruad
# X_train = pd.concat([X_train.loc[not_fraud],X_train.loc[fraud]]) 
# y_train = pd.concat([y_train.loc[not_fraud],y_train.loc[fraud]])

# print("Training Data", len(X_train),len(y_train))


# # Testing Data
# not_fraud = y_test[y_test == 0].index #Get index of rows that are not fraud
# fraud = y_test[y_test == 1].index # #Get index of rows that are fraud
# not_fraud = np.random.choice(not_fraud, len(fraud), replace=False) # Randomly pick not fraud rows without replacement until the len is the same as fruad
# X_test = pd.concat([X_test.loc[not_fraud],X_test.loc[fraud]]) 
# y_test = pd.concat([y_test.loc[not_fraud],y_test.loc[fraud]])

# print(len(X_test),len(y_test))

In [ ]:
# X_train

In [ ]:
# for i in df.columns:
#   print(str(i)+":"+str(df[i].isnull().values.any()))

In [ ]:
# pd.set_option('display.max_columns', 500)
# y_train

## Oversampling

In [ ]:
from imblearn.over_sampling import RandomOverSampler 
from collections import Counter

print('Original dataset shape %s' % Counter(y_train))
ros = RandomOverSampler(random_state=42)
X_train, y_train = ros.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_train))

Original dataset shape Counter({0: 89538, 1: 5302})
Resampled dataset shape Counter({0: 89538, 1: 89538})


## Building the XGB Classifier

In [ ]:
print("Number of Features: ",len(X_train.columns))

Number of Features:  43


In [ ]:
import xgboost as xgb

#MLP = MLPClassifier(hidden_layer_sizes=(50,80,100,70,50,30,20,10,5), max_iter=100, verbose=1, random_state=1)
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42,tree_method='gpu_hist')



In [ ]:
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from scipy.stats import uniform, randint

In [ ]:
def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [ ]:
# params = {
#     "colsample_bytree": uniform(0.7, 0.3),
#     "gamma": uniform(0, 0.5),
#     "learning_rate": uniform(0.03, 0.3), # default 0.1 
#     "max_depth": randint(2, 6), # default 3
#     "n_estimators": randint(100, 150), # default 100
#     "subsample": uniform(0.6, 0.4)
# }

# params['tree_method'] = ['gpu_hist']

# search = RandomizedSearchCV(xgb_model, param_distributions=params, random_state=42, n_iter=300, cv=3, verbose=1, n_jobs=1, return_train_score=True)

# search.fit(X_train, y_train)

# report_best_scores(search.cv_results_, 1)

In [ ]:
{'colsample_bytree': 0.9796084316530052, 'gamma': 0.05125486399533963, 'learning_rate': 0.31116854617093537, 'max_depth': 5, 'n_estimators': 135, 'subsample': 0.6271348236420687, 'tree_method': 'gpu_hist'}


{'colsample_bytree': 0.9796084316530052,
 'gamma': 0.05125486399533963,
 'learning_rate': 0.31116854617093537,
 'max_depth': 5,
 'n_estimators': 135,
 'subsample': 0.6271348236420687,
 'tree_method': 'gpu_hist'}

In [ ]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42,tree_method='gpu_hist', colsample_bytree= 0.9796084316530052, gamma= 0.05125486399533963, learning_rate= 0.31116854617093537, max_depth= 5, n_estimators= 135, subsample= 0.6271348236420687)

## <font color="orange">Training Starts Here ↓ </font>

### The Basic Method

In [ ]:
xgb_model.fit(X_train, y_train)

XGBClassifier(colsample_bytree=0.9796084316530052, gamma=0.05125486399533963,
              learning_rate=0.31116854617093537, max_depth=5, n_estimators=135,
              random_state=42, subsample=0.6271348236420687,
              tree_method='gpu_hist')

### Cross Validation

In [ ]:
df_features = pd.concat([X_train,X_test])
df_target = pd.concat([y_train,y_test])

In [ ]:
df_features

,district,client_catg,region,region_group,creation_date_day,creation_date_month,creation_date_year,no_months_as_client,services_consumed,months_of_service,...,tally_check_false,sum_tally_value,min_tally_value,max_tally_value,mean_tally_value,counter_type,last_year,last_month,last_day,last_day_is_weekday
0,62,11,301,3,27,9,1999,225.943038,1,162.401692,...,0,0,0,0,0.0,1,2018,7,26,1
1,69,11,104,2,18,10,1995,180.537588,1,57.036079,...,0,0,0,0,0.0,1,2010,11,3,1
2,63,11,311,3,14,6,2007,149.752562,1,149.292593,...,0,0,0,0,0.0,1,2019,12,6,1
3,63,11,379,3,28,12,2011,91.730836,1,77.208978,...,0,0,0,0,0.0,1,2019,8,20,1
4,63,11,101,2,26,6,2002,204.160250,2,156.717797,...,0,0,0,0,0.0,2,2019,7,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99761,63,11,312,3,31,5,2004,186.221483,2,168.512700,...,0,0,0,0,0.0,2,2019,12,7,1
24563,62,11,301,3,18,12,2015,37.717407,1,34.399064,...,0,0,0,0,0.0,1,2019,2,8,1
18834,63,11,312,3,29,6,2001,216.973654,2,170.549703,...,0,0,0,0,0.0,2,2019,7,29,1
107,62,11,302,3,26,6,1984,425.372184,1,169.564057,...,0,0,0,0,0.0,1,2019,12,7,1


In [ ]:
from statistics import mean
from sklearn.model_selection import cross_val_score
# scores = cross_val_score(xgb_model, df_features, df_target, cv=5)
scores = cross_val_score(xgb_model, df_features, df_target, cv=5)
print(scores)
print(mean(scores))

[0.87566276 0.88019115 0.8821709  0.87761697 0.86639814]
0.8764079827744232


In [ ]:
mean(scores)

0.8764079827744232

## Prediction of Model

In [ ]:
# plot feature importance
from xgboost import plot_importance
from matplotlib import pyplot
from matplotlib.pyplot import figure

pyplot.rcParams["figure.figsize"] = (14, 7)
pyplot.rcParams["figure.dpi"] = 1200
pyplot.rcParams.update({'font.size':10})
plot_importance(xgb_model)
# pyplot.savefig("export.pdf",bbox_inches='tight')


In [ ]:
importance=xgb_model.get_booster().get_score()

In [ ]:
importance_names = list(importance.keys())
importance = {importance_names[i]: sorted(
    importance.values())[i] for i in range(len(importance_names))}

In [ ]:
importance

{'number_of_counter': 2,
 'number_of_person_counting': 2,
 'sum_total_consumption': 10,
 'last_year': 12,
 'months_of_service': 12,
 'creation_date_year': 16,
 'max_difference_index': 22,
 'min_difference_index': 26,
 'region': 26,
 'mean_reading_remark': 27,
 'last_day': 28,
 'district': 28,
 'services_consumed': 29,
 'mean_counter_statue': 34,
 'sum_difference_index': 36,
 'number_of_instances': 63,
 'creation_date_day': 71,
 'counter_type': 77,
 'mean_difference_index': 85,
 'no_months_as_client': 117,
 'creation_date_month': 118,
 'last_month': 118,
 'mean_total_consumption': 122,
 'max_total_consumption': 125,
 'tally_check_false': 128,
 'mean_tally_value': 133,
 'client_catg': 142,
 'min_tally_value': 150,
 'max_counter_statue': 152,
 'last_day_is_weekday': 154,
 'sum_tally_value': 162,
 'min_total_consumption': 181,
 'sum_counter_coefficient': 189,
 'max_reading_remark': 190,
 'min_counter_statue': 202,
 'tally_check_true': 212,
 'min_reading_remark': 217,
 'max_tally_value': 24

In [ ]:
importance.keys()

dict_keys(['number_of_counter', 'number_of_person_counting', 'sum_total_consumption', 'last_year', 'months_of_service', 'creation_date_year', 'max_difference_index', 'min_difference_index', 'region', 'mean_reading_remark', 'last_day', 'district', 'services_consumed', 'mean_counter_statue', 'sum_difference_index', 'number_of_instances', 'creation_date_day', 'counter_type', 'mean_difference_index', 'no_months_as_client', 'creation_date_month', 'last_month', 'mean_total_consumption', 'max_total_consumption', 'tally_check_false', 'mean_tally_value', 'client_catg', 'min_tally_value', 'max_counter_statue', 'last_day_is_weekday', 'sum_tally_value', 'min_total_consumption', 'sum_counter_coefficient', 'max_reading_remark', 'min_counter_statue', 'tally_check_true', 'min_reading_remark', 'max_tally_value'])

In [ ]:
predicted_target = xgb_model.predict(X_test)


In [ ]:
print("Training set score", xgb_model.score(X_train, y_train))
print("Testing set score", xgb_model.score(X_test, y_test))

Training set score 0.8926489311800576
Testing set score 0.8426206116072527


In [ ]:
from sklearn import metrics
accuracy = metrics.accuracy_score(y_test, predicted_target)
balanced_accuracy = metrics.balanced_accuracy_score(y_test, predicted_target)
precision = metrics.precision_score(y_test, predicted_target)
recall = metrics.recall_score(y_test, predicted_target)
f1 = metrics.f1_score(y_test, predicted_target)
f2 = metrics.fbeta_score(y_test, predicted_target, beta=2.0)
roc_auc = metrics.roc_auc_score(y_test, predicted_target)

print(f"Accuracy: {accuracy}")
print(f"Balanced Accuracy: {balanced_accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 score: {f1}")
print(f"F2 score: {f2}")
print(f"ROC AUC: {roc_auc}")
print(f"Confusion matrix: {metrics.confusion_matrix(y_test, predicted_target)}")
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, predicted_target, labels=xgb_model.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=xgb_model.classes_)
disp.plot()
plt.show()


Accuracy: 0.8426206116072527
Balanced Accuracy: 0.7641278889017876
Precision: 0.21270679827610176
Recall: 0.6757950530035336
F1 score: 0.3235698424447499
F2 score: 0.47079820296633645
ROC AUC: 0.7641278889017876
Confusion matrix: [[32720  5663]
 [  734  1530]]


## References
1. https://xgboost.readthedocs.io/en/stable/